# Training Pipeline

## Installation

In [1]:
%%capture
!pip install unsloth==2025.1.6 comet_ml==3.48.1 evaluate==0.4.3 rouge_score==0.1.2

## Prepare environment

If you want to upload your dataset to Hugging Face, enter your Hugging Token token (it's free).

If you want to use Comet as an experiment tracker enter your Comet API Key (it's free).

In [2]:
import os

from getpass import getpass

hf_token = getpass("Enter your Hugging Face token. Press Enter to skip: ")
enable_hf = bool(hf_token)
print(f"Is Hugging Face enabled? '{enable_hf}'")

comet_api_key = getpass("Enter your Comet API key. Press Enter to skip: ")
enable_comet = bool(comet_api_key)
comet_project_name = "second-brain-course"
print(f"Is Comet enabled? '{enable_comet}'")

if enable_hf:
    os.environ["HF_TOKEN"] = hf_token
if enable_comet:
    os.environ["COMET_API_KEY"] = comet_api_key
    os.environ["COMET_PROJECT_NAME"] = comet_project_name

Enter your Hugging Face token. Press Enter to skip: ··········
Is Hugging Face enabled? 'True'
Enter your Comet API key. Press Enter to skip: ··········
Is Comet enabled? 'True'


## Global variables

Make sure you have an Nvidia GPU active. You can choose it from the **Runtime** tab.

In [3]:
import torch


def get_gpu_info() -> str | None:
    """Gets GPU device name if available.

    Returns:
        str | None: Name of the GPU device if available, None if no GPU is found.
    """
    if not torch.cuda.is_available():
        return None

    gpu_name = torch.cuda.get_device_properties(0).name

    return gpu_name


active_gpu_name = get_gpu_info()

print("GPU type:")
print(active_gpu_name)

GPU type:
NVIDIA L4


In [52]:
dataset_id = input("Enter your Hugging Face dataset_id (which you generated in Lesson 3). Hit enter to use our precomputed version: ") or "pauliusztin/second_brain_course_summarization_task"
print(f"{dataset_id=}")

Enter your Hugging Face dataset_id (which you generated in Lesson 3). Hit enter to use our precomputed version: 
dataset_id='pauliusztin/second_brain_course_summarization_task'


Depending on your GPU type, we must pick different variables, as training in 4bit (QLoRA) takes substantially longer than training in 16bit (LoRA). Thus, if you have a T4 Nivia GPU, which is available in Google's Colab free tier, to avoid waiting an eternity for the fine-tuning to complete, we will train for fewer steps (on T4, we cannot train with LoRA without encountering issues while fine-tuning).

In [4]:
max_seq_length = 4096  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
if active_gpu_name and "T4" in active_gpu_name:
    load_in_4bit = True  # Use 4bit quantization to reduce memory usage.
    max_steps = 25  # Reduce training steps to avoiding waiting too long.
elif active_gpu_name and ("A100" in active_gpu_name or "L4" in active_gpu_name):
    load_in_4bit = False  # Disable 4bit quantization for faster training.
    max_steps = 500  # As we train without 4bit quantization, we can train for more steps without waiting too long.
elif active_gpu_name:
    load_in_4bit = False  # Disable 4bit quantization for faster training.
    max_steps = 250  # As we train without 4bit quantization, we can train for more steps without waiting too long.
else:
    raise ValueError("No Nvidia GPU found.")

max_steps = 25

print("--- Parameters ---")
print(f"{max_steps=}")
print(f"{load_in_4bit=}")
print(f"{dtype=}")


--- Parameters ---
max_steps=5
load_in_4bit=False
dtype=None


## Load LLM using Unsloth

In [5]:
from unsloth import FastLanguageModel
import torch

base_model = "Meta-Llama-3.1-8B-Instruct"  # or unsloth/Qwen2.5-7B-Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=f"unsloth/{base_model}",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2025.1.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Preperation

We now use the Alpaca format to map the instruct dataset into input prompts.

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [7]:
from datasets import load_dataset


alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["instruction"]
    outputs = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN

        texts.append(text)
    return {
        "text": texts,
    }


In [8]:
dataset = load_dataset(dataset_id)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=1,  # Set this for 1 full training run, while commenting out 'max_steps'.
        max_steps=max_steps,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="comet_ml" if enable_comet else "none",
    ),
)

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
15.729 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,467 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 5
 "-____-"     Number of trainable parameters = 41,943,040
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, tensorflow, keras, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/iusztinpaul/second-brain-course/289e4e8aac454c9e9d43bf7f05a8a58e

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Step,Training Loss
1,1.342100
2,1.222200
3,1.294100
4,1.428500
5,1.534800


In [12]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

204.4443 seconds used for training.
3.41 minutes used for training.
Peak reserved memory = 17.178 GB.
Peak reserved memory for training = 1.449 GB.
Peak reserved memory % of max memory = 77.515 %.
Peak reserved memory for training % of max memory = 6.539 %.


### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [36]:
from transformers import TextStreamer


FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer)


def generate_text(instruction, streaming: bool = True, trim_input_message: bool = False):
  message = alpaca_prompt.format(
      instruction,
      "",  # output - leave this blank for generation!
  )
  inputs = tokenizer([message], return_tensors="pt").to("cuda")

  if streaming:
    return model.generate(**inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True)
  else:
    output_tokens = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

    if trim_input_message:
      return output[len(message) :]
    else:
      return output

In [30]:
generate_text(dataset["validation"][0]["instruction"], streaming=True)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
[![](assets/figures/nvidia.svg) Toronto AI Lab  ](https://research.nvidia.com/labs/toronto-ai/)

# 

Align your Latents:High-Resolution Video Synthesis with Latent Diffusion Models

[Andreas Blattmann1 *,†](https://twitter.com/andi_blatt) [Robin Rombach1 *,†](https://twitter.com/robrombach) [Huan Ling2,3,4 *](https://www.cs.toronto.edu/~linghuan/) [Tim Dockhorn2,3,5 *,†](https://timudk.github.io/) [Seung Wook Kim2,3,4](https://seung-kim.github.io/seungkim/) [Sanja Fidler2,3,4](https://www.cs.toronto.edu/~fidler/) [Karsten Kreis2](https://karste

tensor([[128000,  39314,    374,  ...,    220,    510,   7927]],
       device='cuda:0')

In [37]:
generate_text(dataset["validation"][0]["instruction"], streaming=False)

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights\n\n### Input:\n[![](assets/figures/nvidia.svg) Toronto AI Lab  ](https://research.nvidia.com/labs/toronto-ai/)\n\n# \n\nAlign your Latents:High-Resolution Video Synthesis with Latent Diffusion Models\n\n[Andreas Blattmann1 *,†](https://twitter.com/andi_blatt) [Robin Rombach1 *,†](https://twitter.com/robrombach) [Huan Ling2,3,4 *](https://www.cs.toronto.edu/~linghuan/) [Tim Dockhorn2,3,5 *,†](https://timudk.github.io/) [Seung Wook Kim2,3,4](https://seung-kim.github.io/seungkim/) [Sanja Fidler2,3,4](https://www.cs.toronto.edu/~fidler/) [Karsten Kreis2](https://karstenkre

## Evaluate

The last step is to compute some metrics on the validation split to see how well our fine-tuned LLM performs.

In [45]:
import evaluate
import numpy as np

from tqdm import tqdm

rouge = evaluate.load("rouge")


def compute_metrics(predictions: list[str], references: list[str]):
    result = rouge.compute(
        predictions=predictions, references=references, use_stemmer=True
    )
    result["mean_len"] = np.mean([len(p) for p in predictions])

    return {k: round(v, 4) for k, v in result.items()}

In [49]:
predictions = []
references = []

# Take only the first 10 elements to speed up evaluation.
# Keep the whole dataset for an end-to-end evaluation.
validation_samples = dataset["validation"].select(range(10))
for sample in tqdm(validation_samples):
    prediction = generate_text(sample["instruction"], streaming=False, trim_input_message=True)

    predictions.append(prediction)
    references.append(sample["answer"])

predictions[0]

 30%|███       | 3/10 [01:10<02:45, 23.62s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 216.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 71.38 MiB is free. Process 495944 has 22.08 GiB memory in use. Of the allocated memory 21.05 GiB is allocated by PyTorch, and 816.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [40]:
references[0]

'```markdown\n# TL;DR Summary\n\nThe paper presents Video Latent Diffusion Models (Video LDMs) for high-resolution video synthesis, achieving state-of-the-art performance in generating temporally coherent videos. It leverages pre-trained image LDMs, enabling efficient text-to-video generation and personalized content creation.\n```'

In [46]:
validation_metrics = compute_metrics(predictions, references)
print(validation_metrics)

{'rouge1': 0.2769, 'rouge2': 0.1244, 'rougeL': 0.1749, 'rougeLsum': 0.1749, 'mean_len': 1167.5}



## Saving Fine-tuned LLM

The last step is to save the fine-tuned LLM locally and on Hugging Face if the Hugging Face token is available.

In [34]:
from huggingface_hub import HfApi

model_name = f"{base_model}-Second-Brain-Summarization"
print(f"Model name: {model_name}")
model.save_pretrained_merged(
    model_name,
    tokenizer,
    save_method="merged_16bit",
)  # Local saving

if enable_hf:
    api = HfApi()
    user_info = api.whoami(token=hf_token)
    huggingface_user = user_info["name"]
    print(f"Current Hugging Face user: {huggingface_user}")

    model.push_to_hub_merged(
        f"{huggingface_user}/{model_name}",
        tokenizer=tokenizer,
        save_method="merged_16bit",
        token=hf_token,
    )  # Online saving to Hugging Face

Model name: Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.82 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  3%|▎         | 1/32 [00:00<00:07,  4.35it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:36<00:00,  1.13s/it]


Unsloth: Saving tokenizer... Done.
Done.
Current Hugging Face user: pauliusztin


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/pauliusztin/Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization



And we're done! If you have any questions on Unsloth, they have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join their Discord!

Also, you can visit their docs to learn about their supported [models](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
4. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>